Optimisation de l'entrainement pour `focus` 
This is the same function as used in `10_Transfer_learning_what_networks.ipynb`
> ... TODO ... # TODO test without circular padding, with Adam, with no warmstart 

    model = torchvision.models.resnet18(weights=None)

# optimize meta-parameters

In [1]:
# print_gpu_memory()

# print(path_save)
# %ls -l {path}*

# %rm {path} + '.sqlite3'

In [2]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [3]:
model_name = 'resnet101'
dataset = 'bbox' # Select your root between : 'boxes', 'full'
dataset = 'full'

model_filename = f'32_fovea_model_name={model_name}_dataset={dataset}_retrained.pth'
# model_filename = None # do not use resumed net
scan_dicts= {'ateb2' : [0.001, 0.]  }
label_dicts= { 'lr' : 'lr',  'ateb2' : '1 - beta2', 'mutnemom' : 'mutnemom'}

In [ ]:
import retinoto_py as fovea
batch_size = 64
opts_dict = dict(do_fovea=True, model_name=model_name, n_train_stop=50*batch_size, n_val_stop=50*batch_size, n_stop=50*batch_size, )
args = fovea.Params(**opts_dict)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurent	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, do_mask=False, do_fovea=True, rs_min=0.0, rs_max=-5.0, padding_mode='zeros', seed=1998, batch_size=64, num_workers=4, in_memory=True, model_name='resnet101', do_scratch=False, num_epochs=50, n_train_stop=3200, n_val_stop=3200, lr=0.0001, delta1=0.05, delta2=0.001, weight_decay=0.0, shuffle=True, verbose=True)

In [5]:
TRAIN_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'train'
train_dataset, class_to_idx, idx_to_class = fovea.get_dataset(args, TRAIN_DATA_DIR)
train_loader = fovea.get_loader(args, train_dataset)
VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
val_dataset, class_to_idx, idx_to_class = fovea.get_dataset(args, VAL_DATA_DIR)
val_loader = fovea.get_loader(args, val_dataset)

In [6]:
%ls -l {model_filename}

ls: 32_fovea_model_name=resnet101_dataset=full_retrained.pth: No such file or directory


In [ ]:
# subplotpars_scan = SubplotParams(left=0.125, right=.95, bottom=0.25, top=.975)
from dataclasses import asdict, field
# max_threshold = .999
for key in scan_dicts:
   
    print(50*'=')
    print('Scanning along', key, "=", label_dicts[key])
    print(50*'=')
    
    measure_columns = [key, 'accuracy']
    i_loc = 0
    for i_value, value in enumerate(scan_dicts[key]):
        filename = args.data_cache / f'34_optuna_{key}_{i_value}.json'
        if filename.is_file():
            df_scan = fovea.pd.read_json(filename)
        else:
            df_scan = fovea.pd.DataFrame([], columns=measure_columns)
            print('i_value', i_value + 1, ' /', len(scan_dicts[key]), key, '=', value)

            opt = fovea.Params(*opts_dict)

            new_dict = asdict(opt)
            new_dict[key] = value
            new_opt = fovea.Params(**new_dict)
            
            def objective(trial):
                    
                # # new_opt.batch_size = trial.suggest_int('batch_size', 16, 512, log=True, step=1)
                # # new_opt.rs_min = trial.suggest_float('rs_min', -1, 1.)
                # # new_opt.rs_max = trial.suggest_float('rs_max', -7, -4)
                # new_opt.mutnemom = trial.suggest_float('mutnemom', 0.01, 0.2, log=True)
                # scale = 10
                # if new_opt.ateb2>0: new_opt.ateb2 = trial.suggest_float('ateb2', 0.001, 0.9, log=True)
                # new_opt.weight_decay = trial.suggest_float('weight_decay', opt.weight_decay / scale, opt.weight_decay * scale, log=True)
                # new_opt.label_smoothing = trial.suggest_float('label_smoothing', opt.label_smoothing / scale, opt.label_smoothing * scale, log=True)


                scale = 100 
                new_opt.lr_class = trial.suggest_float('lr', opt.lr / scale, opt.lr * scale, log=True)
                # new_opt.im_mean = trial.suggest_float('im_mean', opt.im_mean / scale, opt.im_mean * scale, log=True)
                # new_opt.im_std = trial.suggest_float('im_std', opt.im_std / scale, opt.im_std * scale, log=True)

                # get the architecture of the network
                model = fovea.load_model(args, model_path = model_filename if model_filename.is_file() else None)

                # train and get accuracy on the validation set
                _, model_retrain, df_train = fovea.train_model(args, model=model, train_loader=train_loader, val_loader=val_loader)

                accuracy = df_train['avg_acc_val'].mean()
                
                return accuracy
                
            # 3. Create a study object and optimize the objective function.
            sampler = optuna.samplers.TPESampler(multivariate=True, warn_independent_sampling=False)
            opt_tuna= dict(storage=f"sqlite:///cached_data/34_optuna_.sqlite3", 
                           sampler=sampler, direction='maximize', load_if_exists=True, study_name=f"{key} = {value}")
            study = optuna.create_study(**opt_tuna)
            print(f'Starting optimization for {key} = {value} with params: {new_opt} on {max(150-len(study.trials), 0)} trials / {len(study.trials)} ')
            study.optimize(objective, n_trials=max((150-len(study.trials), 0)), n_jobs=1, show_progress_bar=True)

            print(50*'-.')
            print("Best params: ", study.best_params)
            print("Best value: ", study.best_value)
            print("Best Trial: ", study.best_trial)
            # print("Trials: ", study.trials)
            print(50*'-.')
            df_scan.loc[i_loc] = {key:value, 'accuracy':study.best_value}
            # print_gpu_memory()

            df_scan.to_json(filename, orient='index', indent=2)

        print(df_scan)
        print(50*'=')

    # fig, ax = plt.subplots(figsize=(fig_width, fig_width/phi), subplotpars=subplotpars_scan)
    # gp_scan = df_scan[[key, 'accuracy']].groupby([key])
    # means = gp_scan.mean()
    # errors = gp_scan.std()
    # means.plot.bar(yerr=errors, ax=ax, capsize=4, rot=-60, legend=False, color='r', alpha=.5)
    
    # ax.set_ylabel('Accuracy')
    # ax.set_xlabel(key + ' = ' + label_dicts[key])
    # #ax.set_xscale('log')

    # ax.set_ylim(0, 1)
    # #fig = ax.get_figure()
    # # pos = ax.get_position()
    # # print(pos)
    # plt.show()

Scanning along ateb2 = 1 - beta2


AttributeError: 'PosixPath' object has no attribute 'isfile'

In [ ]:
filename.i